In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
feat_size = 60
context = 5

input_x = tf.placeholder(dtype = tf.float32, shape = [None, feat_size*(2*context +1)])
output_y = tf.placeholder(dtype = tf.float32, shape = [None, 2])

In [3]:
#hidden layer 1

hidden_1 = 512
hidden_1_weights = tf.get_variable( "hidden_1_weights",
                                  shape = [ feat_size*(2*context +1), hidden_1],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_1_bias = tf.get_variable( "hidden_1_bias",
                                    shape = [hidden_1],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_1_dnn = tf.matmul( input_x, hidden_1_weights) + hidden_1_bias
hidden_1_out = tf.nn.relu(hidden_1_dnn)
print(hidden_1_out)

Tensor("Relu:0", shape=(?, 512), dtype=float32)


In [4]:
#hidden layer 2

hidden_2 = 512
hidden_2_weights = tf.get_variable( "hidden_2_weights",
                                  shape = [ hidden_1, hidden_2],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_2_bias = tf.get_variable( "hidden_2_bias",
                                    shape = [hidden_2],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_2_dnn = tf.matmul( hidden_1_out, hidden_2_weights) + hidden_2_bias
hidden_2_out = tf.nn.relu(hidden_2_dnn)
print(hidden_2_out)

Tensor("Relu_1:0", shape=(?, 512), dtype=float32)


In [5]:
#hidden layer 2

hidden_ = 512
hidden_weights = tf.get_variable( "hidden_weights",
                                  shape = [ hidden_2, hidden_],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_bias = tf.get_variable( "hidden_bias",
                                    shape = [hidden_],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_dnn = tf.matmul( hidden_2_out, hidden_weights) + hidden_bias
hidden_out = tf.nn.relu(hidden_dnn)
print(hidden_out)

Tensor("Relu_2:0", shape=(?, 512), dtype=float32)


In [6]:
#bottleneck layer 

bottleneck_dim = 40
bottleneck_weights = tf.get_variable( "bottleneck_weights",
                                  shape = [ hidden_, bottleneck_dim],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

bottleneck_bias = tf.get_variable( "bottleneck_bias",
                                    shape = [bottleneck_dim],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

bottleneck_dnn = tf.matmul( hidden_out, bottleneck_weights) + bottleneck_bias
bottleneck_out = tf.nn.relu(bottleneck_dnn)
print(bottleneck_out)

Tensor("Relu_3:0", shape=(?, 40), dtype=float32)


In [7]:
#hidden layer 4

hidden_4 = 200
hidden_4_weights = tf.get_variable( "hidden_4_weights",
                                  shape = [ bottleneck_dim, hidden_4],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_4_bias = tf.get_variable( "hidden_4_bias",
                                    shape = [hidden_4],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_4_dnn = tf.matmul( bottleneck_out, hidden_4_weights) + hidden_4_bias
hidden_4_out = tf.nn.relu(hidden_4_dnn)
print(hidden_4_out)

Tensor("Relu_4:0", shape=(?, 200), dtype=float32)


In [8]:
#hidden layer 1

hidden_5 = 200
hidden_5_weights = tf.get_variable( "hidden_5_weights",
                                  shape = [ hidden_4, hidden_5],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_5_bias = tf.get_variable( "hidden_5_bias",
                                    shape = [hidden_5],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_5_dnn = tf.matmul( hidden_4_out, hidden_5_weights) + hidden_5_bias
hidden_5_out = tf.nn.relu(hidden_5_dnn)
print(hidden_5_out)

Tensor("Relu_5:0", shape=(?, 200), dtype=float32)


In [9]:
#output layer

final = 2
final_weights = tf.get_variable( "final_weights",
                                  shape = [ hidden_5, final],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

final_bias = tf.get_variable( "final_bias",
                                    shape = [final],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

final_dnn = tf.matmul( hidden_5_out, final_weights) + final_bias
final_out = tf.nn.softmax(final_dnn)
print(final_out)

Tensor("Softmax:0", shape=(?, 2), dtype=float32)


In [10]:
# losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels = output_y, logits = final_dnn))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal( tf.argmax(final_dnn, 1), tf.argmax(output_y, 1))
accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [11]:
#initializing tensors using sessoion
session = tf.InteractiveSession()

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=None)

features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC\\contextData'
development_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC\\devcontextData'

model_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC\\Models'

dev_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC\\Dev_sp'



In [12]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [13]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    indexs = (len(list(os.listdir(test_path))))//2
    
    for folders in range(1, indexs+1):
        features = np.load(os.path.join(test_path, "features"+str(folders)+".npy"))
        labels = np.load(os.path.join(test_path, "labels"+str(folders)+".npy"))
        patches = labels.shape[0]
        patch_prob = session.run(final_out, feed_dict = {input_x:features})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
        
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [14]:
#Training and testing accuracy
session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(0,100):
    for j in range(1,batches+1):  #number of batches
        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        train_step.run(session = session,feed_dict={ input_x:features, output_y:labels })
        
    save_path = saver.save(session, os.path.join(model_path,"Model"+str(i)))
    
    print("iteration  ",(i+1))
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model0 is not in all_model_checkpoint_paths. Manually adding it.
iteration   1
FRR   0.1118421052631579  FAR    0.11157894736842106    mid    5.82786312413
 develop eer   0.1118421052631579

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model1 is not in all_model_checkpoint_paths. Manually adding it.
iteration   2
FRR   0.0868421052631579  FAR    0.08736842105263158    mid    4.95555855585
 develop eer   0.0868421052631579

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model2 is not in all_model_checkpoint_paths. Manually adding it.
iteration   3
FRR   0.08421052631578947  FAR    0.08421052631578947    mid    7.45598241372
 develop eer   0.08421052631578947

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model3 is not in all_model_checkpoint_paths. Manually adding it.
iteration   4
FRR   0.11578947368421053  FAR    0.1157894736

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model31 is not in all_model_checkpoint_paths. Manually adding it.
iteration   32
FRR   0.10526315789473684  FAR    0.10526315789473684    mid    11.9685937324
 develop eer   0.10526315789473684

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model32 is not in all_model_checkpoint_paths. Manually adding it.
iteration   33


KeyboardInterrupt: 

In [18]:
evaluation_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC\\evalcontextData'
eval_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC\\Eval_sp'

with tf.Session() as session:
    for i in range(0,5):
        saver.restore(session, os.path.join(model_path,"Model"+str(i)))
        print("iteration  ",(i+1))
        eer_eval = get_EER(evaluation_path, (i+1), eval_sp_path)
        print(" evaluat eer  ",eer_eval)
        print()

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model0
iteration   1
FRR   0.32280431432973805  FAR    0.3227848101265823    mid    7.66412907359
 evaluat eer   0.32280431432973805

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model1
iteration   2
FRR   0.3489984591679507  FAR    0.3489340439706862    mid    7.30289691482
 evaluat eer   0.3489984591679507

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model2
iteration   3
FRR   0.3443759630200308  FAR    0.3448534310459693    mid    11.4423173757
 evaluat eer   0.3443759630200308

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC\Models\Model3
iteration   4
FRR   0.37057010785824346  FAR    0.37108594270486345    mid    6.85604165258
 evaluat eer   0.37057010785824346

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Des

In [ ]:
train_path = "E:\\siva\\CQCC_CMVN_2017V2\\eval_context_data"
save_path = "E:\\siva\\CQCC_CMVN_2017V2\\BN_DNN_bottle\\eval_BN_relu"

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
session = tf.Session()
saver.restore(session, model_path +str(31))
save_bottleneck_features(train_path, save_path, "train", split = 2)

In [ ]:
def save_bottleneck_features(test_path, save_path, index, split = 1 ):
    print(index)
    
    utterences = (len(list(os.listdir(test_path))))//2        
    factor = int(utterences/split)
    
    for i in range(0, split):
        
        genuine_features = []
        spoof_features = []
        print(i*factor)
        print((i+1)*factor)
        for index in range(i*factor, (i+1)*factor):
            if(index > utterences):
                break
            features = np.load(os.path.join(test_path, "features"+str(index+1)+".npy"))
            labels = np.load(os.path.join(test_path, "labels"+str(index+1)+".npy"))

            bn_features = session.run(bottleneck_out, feed_dict = {input_x : features})

            if(labels[0][0] == 1):
                genuine_features.append(bn_features)
            else:
                spoof_features.append(bn_features)

        genuine_features = np.array(genuine_features)
        spoof_features = np.array(spoof_features)
    
        print(genuine_features.shape)
    #     print(genuine_features[0].shape)
    #     print(genuine_features)
        print(spoof_features.shape)

        savemat(os.path.join(save_path, str(i+1)+"genuine_"+str(index) ), {'genuine':genuine_features})
        savemat(os.path.join(save_path, str(i+1)+"spoof_"+str(index) ), {'spoof':spoof_features})

In [ ]:
with tf.Session() as session:
    for i in range(18,30):
        saver.restore(session, model_path +str(i))
        print("iteration  ",(i+1))
        eer_eval = get_EER(evaluation_path, (i+1), eva_sp_path)
        print(" evaluat eer  ",eer_eval)
        print()